- fit a model to the train images of a 0 and the train images of a 1
- compute and plot the energy distribution for all test images for both models

- in the end, i would like to have a histogram with two distributions. One for ISING model of 1 one for ISING model of 0.

In [24]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../../")
from src.pairwise_fitter import Pairwise_fitter
from pairwise_evaluator import Pairwise_evaluator
import os

In [18]:
# fit the ising model to each digit
def fit_digit(digit,nspin, seed,nsamples,fname = "train-images-unlabeled-{}",\
                inalldir_rel="../../INPUT_all/data/traindata",outdir_rel="../../OUTPUT_mod/data",exe_rel="../../ace_utils/ace"):
    mod = Pairwise_fitter(nsamples,inalldir_rel,fname.format(digit), outdir_rel)
    mod.setup(seed,input_spaced=False)
    mod.fit("ace",exe_rel)




Fitting the models

In [22]:
# digit = 0
# nspin = 121
# seed = 42
# nsamples = 4000
# fit_digit(digit,nspin, seed,nsamples)


In [23]:
# digit = 1
# nspin = 121
# seed = 42
# nsamples = 4000
# fit_digit(digit,nspin, seed,nsamples)

Evaluate both models for test data.

In [36]:

def get_mod_digit(digit,nspin,fname="train-images-unlabeled-{}",outdir="../../OUTPUT_mod/data"):
    fname = fname.format(digit)
    jpath = os.path.join(outdir,fname,fname+"_sep-output-out.j")
    mod = Pairwise_evaluator(jpath, nspin)
    mod.load_ising_paramters()
    return mod

In [38]:
nspin = 121
digits = [0,1]
mods = [get_mod_digit(0,nspin) for i in digits] # both models were stopped after 100 iterations of the ACE algorithm

(7381,)
(7381,)


In [39]:
mods[0]

In [ ]:
# load test data